In [3]:
import os
import trimesh
import numpy as np
from pathlib import Path


In [4]:
root_dir = 'datasets/human_seg'
train_path = os.path.join(root_dir, 'train')
seg_path = os.path.join(root_dir, 'seg_train')

# Save obj files for training in a list
mesh_files = [f for f in os.listdir(train_path) if f.endswith('.obj')]
mesh_files.sort()
print(mesh_files[380])

# Save eseg files for training in a list
# Only shrec_xx.eseg files refer to test data
seg_files = [f for f in os.listdir(seg_path) if f.endswith('.eseg')]
seg_files.sort()
print(seg_files[380])

pair_list = list(zip(mesh_files, seg_files))




mesh_0380.obj
seg_0380.eseg


In [5]:
# Find EDGES starting from the faces of the mesh
def get_edge_faces(faces):
    edge_count = 0
    edge_faces = []
    edge2keys = dict()
    for face_id, face in enumerate(faces):
        for i in range(3):
            cur_edge = tuple(sorted((face[i], face[(i + 1) % 3])))
            if cur_edge not in edge2keys:
                edge2keys[cur_edge] = edge_count
                edge_count += 1
                edge_faces.append(np.array([cur_edge[0], cur_edge[1], -1, -1]))
            edge_key = edge2keys[cur_edge]
            if edge_faces[edge_key][2] == -1:
                edge_faces[edge_key][2] = face_id
            else:
                edge_faces[edge_key][3] = face_id
    return edge_count, np.array(edge_faces), edge2keys


**EXAMPLE**

In [6]:
# EXAMPLE
print(pair_list[0][0])
mesh = trimesh.load(os.path.join(train_path, pair_list[0][0]))
print(mesh.faces)
print("-------------")
edge_count, edge_faces, edge2keys = get_edge_faces(mesh.faces)
print(edge_count)
print("-------------")
print(edge_faces)
print("-------------")
print(edge2keys)
print(edge2keys[(14,735)])

mesh_0000.obj
[[736  10   0]
 [735 732  14]
 [  3   2  37]
 ...
 [351 352 334]
 [487 483 751]
 [751 750 489]]
-------------
2250
-------------
[[  10  736    0 1452]
 [   0   10    0   21]
 [   0  736    0 1449]
 ...
 [ 510  751 1493 1494]
 [ 483  751 1494 1498]
 [ 334  352 1496 1497]]
-------------
{(10, 736): 0, (0, 10): 1, (0, 736): 2, (732, 735): 3, (14, 732): 4, (14, 735): 5, (2, 3): 6, (2, 37): 7, (3, 37): 8, (17, 732): 9, (5, 732): 10, (5, 17): 11, (0, 5): 12, (0, 21): 13, (5, 21): 14, (745, 747): 15, (15, 745): 16, (15, 747): 17, (4, 16): 18, (16, 745): 19, (4, 745): 20, (12, 19): 21, (4, 19): 22, (4, 12): 23, (6, 11): 24, (6, 22): 25, (11, 22): 26, (1, 53): 27, (1, 18): 28, (18, 53): 29, (11, 12): 30, (11, 20): 31, (12, 20): 32, (14, 17): 33, (3, 9): 34, (3, 28): 35, (9, 28): 36, (2, 735): 37, (2, 14): 38, (7, 9): 39, (8, 9): 40, (7, 8): 41, (9, 38): 42, (9, 41): 43, (38, 41): 44, (7, 10): 45, (7, 35): 46, (10, 35): 47, (8, 38): 48, (28, 41): 49, (6, 47): 50, (1, 6): 51, (1, 4

In [38]:
def create_labels_for_mesh(list_obj_seg):
    for mesh_file, seg_file in list_obj_seg:
        mesh = trimesh.load(os.path.join(train_path, mesh_file))
        edge_count, edge_faces, edge2keys = get_edge_faces(mesh.faces)
        seg_list = [line.rstrip() for line in open(os.path.join(seg_path, seg_file), "r")]
        edge_label_pair = list(zip(edge2keys, seg_list))
        print(edge_label_pair)
        return edge_label_pair




In [39]:
edge_label_pairs = create_labels_for_mesh(pair_list)
len(edge_label_pairs)

[((10, 736), '8'), ((0, 10), '8'), ((0, 736), '8'), ((732, 735), '8'), ((14, 732), '8'), ((14, 735), '8'), ((2, 3), '8'), ((2, 37), '8'), ((3, 37), '8'), ((17, 732), '8'), ((5, 732), '8'), ((5, 17), '8'), ((0, 5), '8'), ((0, 21), '8'), ((5, 21), '8'), ((745, 747), '8'), ((15, 745), '8'), ((15, 747), '8'), ((4, 16), '8'), ((16, 745), '8'), ((4, 745), '8'), ((12, 19), '8'), ((4, 19), '8'), ((4, 12), '8'), ((6, 11), '8'), ((6, 22), '8'), ((11, 22), '8'), ((1, 53), '8'), ((1, 18), '8'), ((18, 53), '8'), ((11, 12), '8'), ((11, 20), '8'), ((12, 20), '8'), ((14, 17), '8'), ((3, 9), '8'), ((3, 28), '8'), ((9, 28), '8'), ((2, 735), '8'), ((2, 14), '8'), ((7, 9), '8'), ((8, 9), '8'), ((7, 8), '8'), ((9, 38), '8'), ((9, 41), '8'), ((38, 41), '8'), ((7, 10), '8'), ((7, 35), '8'), ((10, 35), '8'), ((8, 38), '8'), ((28, 41), '8'), ((6, 47), '8'), ((1, 6), '8'), ((1, 47), '8'), ((18, 40), '8'), ((18, 747), '8'), ((40, 747), '8'), ((10, 13), '8'), ((0, 13), '8'), ((13, 35), '8'), ((8, 35), '8'), ((14,

2250

In [40]:
def create_useful_vertices_dict(lati):
    dizionario_vertici = {}

    for (vertice1, vertice2), classe in lati:
        # Aggiungi la classe al dizionario per il primo vertice
        if vertice1 not in dizionario_vertici:
            dizionario_vertici[vertice1] = set(classe)
        else:
            dizionario_vertici[vertice1].add(classe)

        # Aggiungi la classe al dizionario per il secondo vertice
        if vertice2 not in dizionario_vertici:
            dizionario_vertici[vertice2] = set(classe)
        else:
            dizionario_vertici[vertice2].add(classe)

    final_dict = {}
    for key, value in dizionario_vertici.items():
        if len(value) > 1:
            final_dict[key] = value
    return final_dict


In [41]:
useful_vertices_dict = create_useful_vertices_dict(create_labels_for_mesh(pair_list))
print(useful_vertices_dict)

[((10, 736), '8'), ((0, 10), '8'), ((0, 736), '8'), ((732, 735), '8'), ((14, 732), '8'), ((14, 735), '8'), ((2, 3), '8'), ((2, 37), '8'), ((3, 37), '8'), ((17, 732), '8'), ((5, 732), '8'), ((5, 17), '8'), ((0, 5), '8'), ((0, 21), '8'), ((5, 21), '8'), ((745, 747), '8'), ((15, 745), '8'), ((15, 747), '8'), ((4, 16), '8'), ((16, 745), '8'), ((4, 745), '8'), ((12, 19), '8'), ((4, 19), '8'), ((4, 12), '8'), ((6, 11), '8'), ((6, 22), '8'), ((11, 22), '8'), ((1, 53), '8'), ((1, 18), '8'), ((18, 53), '8'), ((11, 12), '8'), ((11, 20), '8'), ((12, 20), '8'), ((14, 17), '8'), ((3, 9), '8'), ((3, 28), '8'), ((9, 28), '8'), ((2, 735), '8'), ((2, 14), '8'), ((7, 9), '8'), ((8, 9), '8'), ((7, 8), '8'), ((9, 38), '8'), ((9, 41), '8'), ((38, 41), '8'), ((7, 10), '8'), ((7, 35), '8'), ((10, 35), '8'), ((8, 38), '8'), ((28, 41), '8'), ((6, 47), '8'), ((1, 6), '8'), ((1, 47), '8'), ((18, 40), '8'), ((18, 747), '8'), ((40, 747), '8'), ((10, 13), '8'), ((0, 13), '8'), ((13, 35), '8'), ((8, 35), '8'), ((14,

In [28]:
def delete_useless_vertices_from_list(edge_label_pairs, useful_vertices_dict):
    for (v1,v2), label in edge_label_pairs:
        if v1 not in useful_vertices_dict and v2 not in useful_vertices_dict:
            edge_label_pairs.remove(((v1,v2),label))

In [44]:
useful_pairs = [elem for elem in edge_label_pairs if elem[0][0] in useful_vertices_dict or elem[0][1] in useful_vertices_dict]
print(useful_pairs)

[((1, 53), '8'), ((18, 53), '8'), ((18, 48), '8'), ((40, 48), '8'), ((48, 53), '8'), ((47, 53), '8'), ((8, 56), '8'), ((45, 56), '8'), ((38, 56), '8'), ((25, 43), '8'), ((26, 43), '8'), ((43, 719), '8'), ((34, 49), '8'), ((40, 49), '8'), ((33, 56), '7'), ((33, 45), '8'), ((26, 39), '8'), ((29, 39), '8'), ((39, 43), '8'), ((53, 63), '8'), ((47, 63), '8'), ((36, 45), '8'), ((35, 36), '8'), ((31, 36), '8'), ((41, 54), '8'), ((38, 54), '8'), ((42, 49), '8'), ((52, 72), '7'), ((39, 52), '8'), ((39, 72), '8'), ((43, 72), '8'), ((31, 43), '8'), ((54, 56), '8'), ((36, 43), '8'), ((28, 51), '8'), ((41, 51), '8'), ((63, 713), '8'), ((47, 713), '8'), ((28, 704), '8'), ((29, 704), '8'), ((48, 60), '8'), ((40, 60), '8'), ((49, 60), '8'), ((52, 62), '7'), ((39, 62), '7'), ((59, 60), '7'), ((48, 59), '7'), ((49, 50), '8'), ((50, 710), '7'), ((49, 710), '7'), ((44, 55), '8'), ((42, 55), '8'), ((51, 54), '8'), ((39, 704), '8'), ((55, 58), '8'), ((60, 710), '7'), ((62, 704), '7'), ((36, 66), '7'), ((43,

In [51]:
from collections import defaultdict

def trova_componenti_connesse(lati):
    # Crea un dizionario per rappresentare il grafo
    grafo = defaultdict(None)
    
    # Popola il grafo con i lati
    for (v1, v2), _ in lati:
        grafo[v1].append(v2)
        grafo[v2].append(v1)
    
    # Funzione di ricerca (DFS) per esplorare una componente connessa
    def dfs(v, visitato, componente):
        visitato.add(v)
        componente.append(v)
        for vicino in grafo[v]:
            if vicino not in visitato:
                dfs(vicino, visitato, componente)

    # Variabili per tracciare i vertici visitati e le componenti connesse
    visitato = set()
    componenti = []
    
    # Trova tutte le componenti connesse
    for vertice in grafo:
        if vertice not in visitato:
            componente = []
            dfs(vertice, visitato, componente)
            componenti.append(componente)
    
    return componenti

# # Esempio di utilizzo
lati = [((10, 736), '8'), ((0, 10), '8'), ((0, 736), '3'), ((500, 600), '2'), ((600, 700), '2')]
componenti_connesse = trova_componenti_connesse(lati)

# Stampa le componenti connesse
for idx, componente in enumerate(componenti_connesse):
    print(f"Componente connessa {idx + 1}: {componente}")


KeyError: 10

In [22]:
# Generiamo una mesh di esempio (insieme di punti 3D)
# Ogni punto ha coordinate (x, y, z)
mesh = trimesh.load(os.path.join(train_path, mesh_files[194]))

vertici = mesh.vertices
vertici -= np.mean(vertici, axis=0)
vertici /= np.max(np.linalg.norm(vertici, axis=1))
import plotly.graph_objs as go
import numpy as np


# Separiamo i vertici con x positiva e negativa
vertici_x_positiva = vertici[vertici[:, 2] > 0]
vertici_x_negativa = vertici[vertici[:, 2] <= 0]

# Crea il grafico per i vertici con x positiva (rosso)
trace1 = go.Scatter3d(
    x=vertici_x_positiva[:, 0],
    y=vertici_x_positiva[:, 1],
    z=vertici_x_positiva[:, 2],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='x > 0'
)

# Crea il grafico per i vertici con x negativa (blu)
trace2 = go.Scatter3d(
    x=vertici_x_negativa[:, 0],
    y=vertici_x_negativa[:, 1],
    z=vertici_x_negativa[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue'),
    name='x <= 0'
)

# Configurazione del layout
layout = go.Layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Mesh Vertices Color by X Coordinate'
)

# Combina i due grafici
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Mostra il grafico interattivo
fig.show()

